In [2]:
import os
from experiment.helpers import *

def replace_with_values(hashsize=8, iterations=1000, epsilon=0.5, epsilon_mod=5, mod_precision=10, verbose = 0):
    props_and_values = {
        "HASHSIZE": hashsize,
        "ITERATIONS": iterations,
        "EPSILON" : '{0:f}'.format(epsilon),
        "EPSILON_MOD": int(epsilon_mod),
        "MOD_PRECISION" : mod_precision
    }

    x86_kalman_dir = os.path.join(os.path.dirname(os.getcwd()),"KalmanC++")
    x86_kalman_hash_header = os.path.join(x86_kalman_dir, "HashTable", "hashtable.h")

    hashtable_header_file_path = x86_kalman_hash_header
    hashtable_header_file_clone_path = os.path.join(x86_kalman_dir, "Originals", "hashtable.h")
    replace_define_value_in_file(hashtable_header_file_clone_path, hashtable_header_file_path, props_and_values)

    if verbose >= 2: print("File '" + hashtable_header_file_path + "' now contains:")
    if verbose >= 2: print_file(hashtable_header_file_path, "#define HASHSIZE")
    if verbose >= 2: print_file(hashtable_header_file_path, "#define EPSILON")
    if verbose >= 2: print_file(hashtable_header_file_path, "#define EPSILON_MOD")
    if verbose >= 2: print_file(hashtable_header_file_path, "#define MOD_PRECISION")
    if verbose >= 2: print_file(hashtable_header_file_path, "#define ITERATIONS")

In [3]:
replace_with_values(hashsize=8, iterations=1000, epsilon=0.5, epsilon_mod=5000, mod_precision=10000, verbose=2)
compile_hash()
print("")
run_hash(verbose=2)

File 'C:\Users\Dragos\Dropbox\RISE\Git\KalmanC++\HashTable\hashtable.h' now contains:
#define HASHSIZE 8
#define EPSILON 0.500000
#define EPSILON_MOD 5000
#define MOD_PRECISION 10000
#define ITERATIONS 1000

Epsilon: 0.500000
Epsilon mod: 5000
Mod precision: 10000
Hash size: 8

HashTable population: 4 / 8
No. of times found: 250
No. of times stored: 750


In [7]:
def get_from_debug(path = "C:\\Users\\Dragos\\Dropbox\\RISE\\Git\\KalmanC++\\debug.txt", verbose=0):
    epsilon = 0
    epsilon_mod = 0
    mod_precision = 0
    hash_size = 0
    utilizied = 0
    found = 0
    stored = 0
    
    for line in open(path, "r"):
        if ("Epsilon:" in line):
            epsilon = float(line.split(" ")[-1])
        elif ("Epsilon mod" in line):
            epsilon_mod = int(line.split(" ")[-1])
        elif ("Mod precision" in line):
            mod_precision = int(line.split(" ")[-1])
        elif ("Hash size" in line):
            hash_size = int(line.split(" ")[-1])
        elif ("HashTable population" in line):
            utilized = int(line.split(" ")[-3])
        elif ("found" in line):
            found = int(line.split(" ")[-1])
        elif ("stored" in line):
            stored = int(line.split(" ")[-1])
        if verbose>0:print(line, end='')
            
    return (epsilon, epsilon_mod, mod_precision, hash_size, utilized, found, stored)

In [8]:
get_from_debug(verbose=1)

Epsilon: 0.750000
Epsilon mod: 75
Mod precision: 100
Hash size: 8

HashTable population: 4 / 8
No. of times found: 431
No. of times stored: 569


(0.75, 75, 100, 8, 4, 431, 569)

In [36]:
import sys
import numpy as np

hash_sizes = [120, 100, 64, 32, 16, 8]
epsilon=np.arange(1.0, 0.8, -0.01) #[1.0, 0.99, 0.98, 0.97 0.9, 0.85, 0.84, 0.83, 0.82, 0.81, 0.8, 0.75, 0.5, 0.25, 0.1, 0.01, 0.001, 0.0001, 0.00001]
mod_precision=[10,100,1000,10000,100000]
iterations = 2888

max_found = 0
min_found = 9999
max_utilized = 0

selected_hash_size = 8
selected_epsilon = 0.1
selected_mod_precision = 10
selected_epsilon_mod = 0

selected_utilized = 0
selected_stored = 0
selected_found = 0

for h in hash_sizes:
    for e in epsilon:
        for m in mod_precision:
            if e*m - int(e*m) > 0:
                continue # If fraction part is not 0
            replace_with_values(hashsize=h, epsilon=e, mod_precision=m, epsilon_mod=e*m, iterations=iterations)
            compile_hash()
            #sleep(0.001)
            run_hash(verbose=0)
            #sleep(0.5)
            (e_, em_, m_, h_, utilized, found, stored) = get_from_debug()
            try:
                assert(e == e_)
            except AssertionError:
                #print("e != e_: {0} -- {1}".format(e, e_))
                pass
            try:
                assert(em_ == e*m)
            except AssertionError:
                print("em != e*m: {0} -- {1}".format(em_, e*m))
            assert(m == m_)
            assert(h == h_)
            #if utilized_ > max_utilized:
            #if found > max_found: # and balance < min_balance:
            #if balance < min_balance:
            if found > (iterations / 2) and found < ((iterations / 2) + 100) and found > max_found:
                selected_hash_size = h
                selected_epsilon = e__
                selected_mod_precision = m
                selected_epsilon_mod = e*m
                selected_utilized = utilized
                
                selected_stored = stored
                selected_found = found
                
                max_found = found
                min_found = found
                min_utilized = utilized
            
            sys.stdout.write("\r(Hash, Eps, Mod, EpsMod | Utilized, Stored, Found): Running = ({0}, {1}, {2}, {3} | {4}, {5}, {6}); Selected = ({7}, {8}, {9}, {10}) | {11}, {12}, {13})".format( \
                h,e,m,e*m, utilized, stored, found, selected_hash_size, selected_epsilon, selected_mod_precision, selected_epsilon_mod, selected_utilized, selected_stored, selected_found))
            sys.stdout.flush()
                
print("\rHash size: {0}, Epsilon: {1}, Mod precision: {2}, Epsilon mod: {3}".format(selected_hash_size, selected_epsilon, selected_mod_precision, selected_epsilon_mod))

print("Other details: ")
print("- Stored: {0}".format(selected_stored))
print("- Found: {0}".format(selected_found))


Hash size: 120, Epsilon: 0.94, Mod precision: 100, Epsilon mod: 94.0
Other details: 
- Stored: 1345
- Found: 1543


In [63]:
replace_with_values(hashsize=8, iterations=1000, epsilon=0.1, epsilon_mod=7, mod_precision=10, verbose=2)
compile_hash()
print("")
run_hash(verbose=2)

File 'C:\Users\Dragos\Dropbox\RISE\Git\KalmanC++\HashTable\hashtable.h' now contains:
#define HASHSIZE 8
#define EPSILON 0.100000
#define EPSILON_MOD 7
#define MOD_PRECISION 10
#define ITERATIONS 1000

Epsilon: 0.100000
Epsilon mod: 7
Mod precision: 10
Hash size: 8

HashTable population: 4 / 8
No. of times found: 2
No. of times stored: 998
